In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [3]:
PATH = Path('/content/data/')

In [4]:
np.random.seed(42)

In [5]:
src = (ImageList.from_csv(PATH,'train.csv',folder='train',suffix='.jpg')
       .split_by_rand_pct(0.2,42)
       .label_from_df()
      )

In [6]:
tfms = get_transforms(flip_vert=True)

In [7]:
def getdata(sz,bs):
    return src.transform(tfms=tfms,size=sz).databunch(bs=bs).normalize(imagenet_stats)

In [8]:
sz = 500
bs = 64

In [9]:
data = getdata(sz,bs)

In [10]:
arch = models.squeezenet1_1

In [11]:
learn = cnn_learner(data,arch,metrics=[accuracy])

In [12]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
0,1.687504,0.884368,0.776429,07:01
1,0.719697,0.420356,0.884035,07:18
2,0.473882,0.275704,0.922330,07:10
3,0.374288,0.244867,0.931499,07:18


In [15]:
learn.model_dir = '/content/gdrive/My Drive/GardeNerd/squeezenet_models'

In [16]:
learn.save('stage-1')

# Submission

In [ ]:
sample_df = pd.read_csv(PATH/'sample_submission.csv')
learn.data.add_test(ImageList.from_df(sample_df,PATH,folder='test',suffix='.jpg'))

#preds,y = learn.get_preds(ds_type=DatasetType.Test)
preds,y = learn.TTA(ds_type=DatasetType.Test)
sample_df.category = preds.argmax(1) + 1
sample_df.to_csv('submission.csv',index=False)